In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
os.environ['PYTHONPATH'] = os.environ['PYTHONPATH'] + 'C:\\Users\\rdpuser\\Documents\\Github\\parkit;'
sys.path.append('C:\\Users\\rdpuser\\Documents\\Github\\parkit')

In [ ]:
import logging
import multiprocessing
import queue

import parkit as pk

logger = logging.getLogger(__name__)

pk.import_site('C:\\Users\\rdpuser\\Desktop\\test')

In [ ]:
# Queue

In [ ]:
q1 = queue.Queue()
q2 = pk.Queue('memory')
q3 = multiprocessing.Queue()
q4 = pk.Queue()

In [ ]:
%%timeit
for i in range(1000):
    q1.put_nowait(i)
for i in range(1000):
    q1.get_nowait()

In [ ]:
%%timeit
with pk.transaction(q2):
    for i in range(1000):
        q2.put(i)
    for i in range(1000):
        q2.get()

In [ ]:
%%timeit
for i in range(1000):
    q2.put(i)
for i in range(1000):
    q2.get()

In [ ]:
%%timeit
for i in range(1000):
    q3.put(i)
for i in range(1000):
    q3.get()

In [ ]:
# Array

In [ ]:
a1 = list()
a2 = pk.Array('memory')

In [ ]:
%%timeit
for i in range(10000):
    a1.append(i)
a1.clear()

In [ ]:
%%timeit
with pk.transaction(a2):
    for i in range(10000):
        a2.append(i)
    a2.clear()

In [ ]:
%%timeit
for i in range(10000):
    a2.append(i)
a2.clear()

In [ ]:
# Dict

In [ ]:
d1 = dict()
d2 = pk.Dict('memory')

In [ ]:
%%timeit
for i in range(10000):
    d1[i] = i
d1.clear()

In [ ]:
%%timeit
with pk.transaction(d2):
    for i in range(10000):
        d2[i] = i
    d2.clear()

In [ ]:
%%timeit
for i in range(10000):
    d2[i] = i
d2.clear()